Urban Data Science & Smart Cities <br>
URSP688Y <br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

[<img src="https://colab.research.google.com/assets/colab-badge.svg"> Clean version](https://colab.research.google.com/github/ncsg/ursp688y_sp2024/blob/main/demos/demo09/demo09.ipynb)

[<img src="https://colab.research.google.com/assets/colab-badge.svg"> Modified in class](https://colab.research.google.com/drive/1yWxf4Jvh0vEEByjydrp4B4GzwHSGUYr0?usp=sharing)

# Demo 7 - Geospatial Data

- Coordinate systems
- Points from XY
- Loading shapefiles and geojsons
- Proximity analysis
    - Measuring distance
    - Buffering
- Overlay analysis
- Spatial joining

In [1]:
import os
import pandas as pd
import geopandas as gpd

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Set the working directory
# # You will need to change this to your own folder on Google Drive
# os.chdir('/content/drive/MyDrive/Teaching/URSP688Y Spring 2024/demos/week07')

## Geospatial data fundamentals
- Basic geometry types:
    - Points
    - Linestrings
    - Polygons

<img alt="points, lines, and polygons" width=500 src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png">

- Spatial analysis (with vector data) is essentially just Euclidean geometry (remember the Pythagorean theorem?)

<img alt="pythagorean theorem" width=500 src="https://www.katesmathlessons.com/uploads/1/6/1/0/1610286/published/how-to-use-the-pythagorean-theorem-to-find-distance-between-points-on-coordinate-plane-2.png?1595954050">

## Geospatial data are everywhere

DC affordable housing: https://opendata.dc.gov/datasets/DCGIS::affordable-housing/about

### Read CSV

In [ ]:
df = pd.read_csv('Affordable_Housing.csv')

In [ ]:
df.head()

### Coordinate Systems
- Latitude and longitude
- Map projections

### Convert XY coordinates to points

Here are some quick references for the package (GeoPandas), function (`points_from_xy`), and Class (a GeoDataFrame) we'll use to do this. We can also quickly make a very simple map with the `plot` method.

- [GeoPandas](https://geopandas.org/en/stable/getting_started/introduction.html)

- [`points_from_xy`](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html)

- [GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html)

- [`plot`](https://geopandas.org/en/stable/docs/user_guide/mapping.html) method



In [ ]:
# Geometry array without coordinate system defined
geoms = gpd.points_from_xy(df['X'], df['Y'])
geoms

In [ ]:
# Geometry array with coordinate system defined (4326 is the EPSG code for WGS84, the most common latitude-longitude coordinate system) 
geoms = gpd.points_from_xy(df['X'], df['Y'], crs=4326)
geoms.crs

### Create a GeoDataFrame from a DataFrame plus a geometry column

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=geoms, crs=4326)

In [ ]:
gdf.plot()

In [ ]:
gdf.crs

### Read Shapefile 

In [ ]:
gdf = gpd.read_file('Affordable_Housing.shp')

In [ ]:
gdf.plot()

In [ ]:
gdf.crs

### Read GeoJSON

In [ ]:
gdf = gpd.read_file('affordable_housing.geojson')

In [ ]:
gdf.plot()

In [ ]:
gdf.crs

## Proximity

### How far is each affordable housing project from Metro Center?

Basic pseudocode steps:

1. Get location of Metro Center

2. For each housing project, how far is it from Metro Center?

- For loop approach:
    - Get housing projects as a dataframe (list of points)
        - Reproject into a coordinate system where we can measure distance (e.g., UTM)
    - Get location of Metro Center as a point
        - Reproject into a coordinate system where we can measure distance (e.g., UTM)
    - For each housing project:
        - Measure distance to the metro center point

In [2]:
# Store the EPSG code for utm18, a coordinate system in which we can reliably measure distance
UTM18 = 26918
# And for WGS84
WGS84 = 4326

# Load the affordable housing projects as a dataframe
gdf = gpd.read_file('affordable_housing.geojson')
# Project into UTM18
gdf = gdf.to_crs(epsg=UTM18)

# Load the location of metro center (lookup up lon and lat from Google Maps)
metro_center = [38.8985198, -77.032774]
# Convert into a geoseries and project into UTM18
metro_center = gpd.points_from_xy([-77.032774], [38.8985198], crs=WGS84)
metro_center = metro_center.to_crs(epsg=UTM18)
# Get just the first point from the series
metro_center = metro_center[0]

# Initiate a list to store collected distances 
dists = []
for point in gdf['geometry']:
    dist = point.distance(metro_center)
    dists.append(dist)

gdf['utm_dist_to_metro_center'] = dists

gdf[['PROJECT_NAME','utm_dist_to_metro_center']].head()

,PROJECT_NAME,utm_dist_to_metro_center
0,5201 Hayes Street (Deanwood Hills),9206.087612
1,5333 Connecticut Ave NW,7707.830052
2,5741 Colorado Avenue NW Tenant Association,6774.107424
3,62nd Street Apartments (w/ PADD) - Phase II (a...,10495.218911
4,6925-6929 Georgia Avenue,8428.154639


- Vectorized approach
    - **Pick up here in week 9**

#### Haversine formula
The [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) can be used to calculate distances along the surface of a sphere, approximating distances between latitude and longitude points.

Here's a function implementing it in Python:

In [3]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)

    From https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

We can rewrite our code using this function instead of the distance method we used previously:

In [13]:
# Load the location of metro center (lookup up lon and lat from Google Maps)
metro_center_lon = -77.032774
metro_center_lat = 38.8985198

# Make a copy of the affordable housing points in WGS84
gdf_wgs84 = gdf.to_crs(WGS84)

# Initiate a list to store collected distances 
dists = []
for point in gdf_wgs84['geometry']:
    dist = haversine(metro_center_lon, metro_center_lat, point.x, point.y)
    dists.append(dist)

gdf['haversine_dist_to_metro_center'] = dists

gdf[['PROJECT_NAME','utm_dist_to_metro_center', 'haversine_dist_to_metro_center']].head()

,PROJECT_NAME,utm_dist_to_metro_center,haversine_dist_to_metro_center
0,5201 Hayes Street (Deanwood Hills),9206.087612,9.184001
1,5333 Connecticut Ave NW,7707.830052,7.714206
2,5741 Colorado Avenue NW Tenant Association,6774.107424,6.785241
3,62nd Street Apartments (w/ PADD) - Phase II (a...,10495.218911,10.470167
4,6925-6929 Georgia Avenue,8428.154639,8.441898


## Overlay/Spatial Join

### How many projects are within 5 km of Metro Center?

- [`buffer`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html) method

### How many projects are in each census tract?

- [Tract geometries from Census Reporter](https://censusreporter.org/data/table/?table=B01003&geo_ids=16000US1150000,140|16000US1150000&primary_geo_id=16000US1150000)

- [`sjoin`](https://geopandas.org/en/stable/gallery/spatial_joins.html) function

### How many units are in each census tract?